## Using httr to interact with APIs directly
If an API client doesn't exist, it's up to you to communicate directly with the API. But don't worry, the package `httr` makes this really straightforward. In this chapter you'll learn how to make web requests from R, how to examine the responses you get back and some best practices for doing this in a responsible way.

### GET requests in practice
To start with you're going to make a GET request. As discussed in the video, this is a request that asks the server to give you a particular piece of data or content (usually specified in the URL). These make up the majority of the requests you'll make in a data science context, since most of the time you'll be getting data from servers, not giving it to them.

To do this you'll use the httr package, written by Hadley Wickham (of course), which makes HTTP requests extremely easy. You're going to make a very simple GET request, and then inspect the output to see what it looks like.

In [2]:
# Load the httr package
library(httr)

# Make a GET request to http://httpbin.org/get
get_result <- GET("http://httpbin.org/get")

# Print it to inspect it
print(get_result)

Warning message:
"package 'httr' was built under R version 3.6.3"

Response [http://httpbin.org/get]
  Date: 2021-01-13 23:52
  Status: 200
  Content-Type: application/json
  Size: 365 B
{
  "args": {}, 
  "headers": {
    "Accept": "application/json, text/xml, application/xml, */*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "libcurl/7.59.0 r-curl/3.3 httr/1.4.2", 
    "X-Amzn-Trace-Id": "Root=1-5fff87a9-01cc2490644e8dab6bfb86ce"
  }, 
  "origin": "47.63.187.254", 
...


### POST requests in practice
Next we'll look at POST requests, also made through httr, with the function (you've guessed it) POST(). Rather than asking the server to give you something, as in GET requests, a POST request asks the server to accept something from you. They're commonly used for things like file upload, or authentication. As a result of their use for uploading things, POST() accepts not just a url but also a body argument containing whatever you want to give to the server.

You'll make a very simple POST request, just uploading a piece of text, and then inspect the output to see what it looks like.

In [3]:
# Make a POST request to http://httpbin.org/post with the body "this is a test"
post_result <-POST(url = "http://httpbin.org/post", body = "this is a test")

# Print it to inspect it
post_result

Response [http://httpbin.org/post]
  Date: 2021-01-10 10:05
  Status: 200
  Content-Type: application/json
  Size: 472 B
{
  "args": {}, 
  "data": "this is a test", 
  "files": {}, 
  "form": {}, 
  "headers": {
    "Accept": "application/json, text/xml, application/xml, */*", 
    "Accept-Encoding": "gzip, deflate", 
    "Content-Length": "14", 
    "Host": "httpbin.org", 
...

### Extracting the response
Making requests is all well and good, but it's also not why you're here. What we really want to do is get the data the server sent back, which can be done with httr's content() function. You pass it an object returned from a GET (or POST, or DELETE, or...) call, and it spits out whatever the server actually sent in an R-compatible structure.

We're going to demonstrate that now, using a slightly more complicated URL than before - in fact, using a URL from the Wikimedia pageviews system you dealt with through the pageviews package, which is stored as url. Without looking too much at the structure for the time being (we'll get to that later) this request asks for the number of pageviews to the English-language Wikipedia's "Hadley Wickham" article on 1 and 2 January 2017.

In [4]:
# url 

url = "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia.org/all-access/all-agents/Hadley_Wickham/daily/20170101/20170102"

# Make a GET request to url and save the results
pageview_response <- GET(url)

# Call content() to retrieve the data the server sent back
pageview_data <- content(pageview_response)

# Examine the results with str()
str(pageview_data)

List of 1
 $ items:List of 2
  ..$ :List of 7
  .. ..$ project    : chr "en.wikipedia"
  .. ..$ article    : chr "Hadley_Wickham"
  .. ..$ granularity: chr "daily"
  .. ..$ timestamp  : chr "2017010100"
  .. ..$ access     : chr "all-access"
  .. ..$ agent      : chr "all-agents"
  .. ..$ views      : int 45
  ..$ :List of 7
  .. ..$ project    : chr "en.wikipedia"
  .. ..$ article    : chr "Hadley_Wickham"
  .. ..$ granularity: chr "daily"
  .. ..$ timestamp  : chr "2017010200"
  .. ..$ access     : chr "all-access"
  .. ..$ agent      : chr "all-agents"
  .. ..$ views      : int 86


### Handling http failures
As mentioned, HTTP calls can go wrong. Handling that can be done with httr's http_error() function, which identifies whether a server response contains an error.

If the response does contain an error, calling http_error() over the response will produce TRUE; otherwise, FALSE. You can use this for really fine-grained control over results. For example, you could check whether the request contained an error, and (if so) issue a warning and re-try the request.

For now we'll try something a bit simpler - issuing a warning that something went wrong if http_error() returns TRUE, and printing the content if it doesn't.

In [5]:
fake_url <- "http://google.com/fakepagethatdoesnotexist"

# Make the GET request
request_result <- GET(fake_url)

# Check request_result
if(http_error(request_result)){
    warning("The request failed")
} else {
    content(request_result)
}

Warning message in eval(expr, envir, enclos):
"The request failed"

### Constructing queries (Part I)
As briefly discussed in the previous video, the actual API query (which tells the API what you want to do) tends to be in one of the two forms. The first is directory-based, where values are separated by / marks within the URL. The second is parameter-based, where all the values exist at the end of the URL and take the form of key=value.

Constructing directory-based URLs can be done via paste(), which takes an unlimited number of strings, along with a separator, as sep. So to construct http://swapi.co/api/vehicles/12 we'd call:

paste("http://swapi.co", "api", "vehicles", "12", sep = "/")
Let's do that now! We'll cover parameter-based URLs later. In the mean time we can play with SWAPI, mentioned above, which is an API chock full of star wars data. This time, rather than a vehicle, we'll look for a person.

In [7]:
# Construct a directory-based API URL to `http://swapi.co/api`,
# looking for person `1` in `people`
directory_url <- paste("http://swapi.co/api", "people", "1", sep = "/")

# Make a GET call with it
result <- GET(directory_url)
result

Response [http://swapi.co/api/people/1]
  Date: 2021-01-10 11:41
  Status: 404
  Content-Type: text/html; charset=UTF-8
  Size: 24 B
<h1>404: Not Found</h1>

### Constructing queries (Part II)
As mentioned (albeit briefly) in the last exercise, there are also parameter based URLs, where all the query values exist at the end of the URL and take the form of key=value - they look something like http://fakeurl.com/foo.php?country=spain&food=goulash

Constructing parameter-based URLs can also be done with paste(), but the easiest way to do it is with GET() and POST() themselves, which accept a query argument consisting of a list of keys and values. So, to continue with the food-based examples, we could construct fakeurl.com/api.php?fruit=peaches&day=thursday with:

GET("fakeurl.com/api.php", query = list(fruit = "peaches", day = "thursday"))
In this exercise you'll construct a call to https://httpbin.org/get?nationality=americans&country=antigua.

In [8]:
# Create list with nationality and country elements
query_params <- list(nationality = "americans", 
    country = "antigua")
    
# Make parameter-based call to httpbin, with query_params
parameter_response <- GET("https://httpbin.org/get", query = query_params)

# Print parameter_response
print(parameter_response)

Response [https://httpbin.org/get?nationality=americans&country=antigua]
  Date: 2021-01-10 11:53
  Status: 200
  Content-Type: application/json
  Size: 465 B
{
  "args": {
    "country": "antigua", 
    "nationality": "americans"
  }, 
  "headers": {
    "Accept": "application/json, text/xml, application/xml, */*", 
    "Accept-Encoding": "gzip, deflate", 
    "Host": "httpbin.org", 
    "User-Agent": "libcurl/7.59.0 r-curl/3.3 httr/1.4.2", 
...


### Using user agents
As discussed in the video, informative user-agents are a good way of being respectful of the developers running the API you're interacting with. They make it easy for them to contact you in the event something goes wrong. I always try to include:

My email address;

1. A URL for the project the code is a part of, if it's got a URL.
2. Building user agents is done by passing a call to user_agent() into the GET() or POST() request; something like:

GET("http://url.goes.here/", user_agent("somefakeemail@domain.com http://project.website"))
In the event you don't have a website, a short one-sentence description of what the project is about serves pretty well.

In [10]:
# Do not change the url
url <- "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents/Aaron_Halfaker/daily/2015100100/2015103100"

# Add the email address and the test sentence inside user_agent(). Include a user agent that has a fake email address 
# "my@email.address" followed by the sentence "this is a test".
server_response <- GET(url, user_agent(paste("my@email.address","this is a test", sep= " ")))

$items
$items[[1]]
$items[[1]]$project
[1] "en.wikipedia"

$items[[1]]$article
[1] "Aaron_Halfaker"

$items[[1]]$granularity
[1] "daily"

$items[[1]]$timestamp
[1] "2015100100"

$items[[1]]$access
[1] "all-access"

$items[[1]]$agent
[1] "all-agents"

$items[[1]]$views
[1] 8


$items[[2]]
$items[[2]]$project
[1] "en.wikipedia"

$items[[2]]$article
[1] "Aaron_Halfaker"

$items[[2]]$granularity
[1] "daily"

$items[[2]]$timestamp
[1] "2015100200"

$items[[2]]$access
[1] "all-access"

$items[[2]]$agent
[1] "all-agents"

$items[[2]]$views
[1] 3


$items[[3]]
$items[[3]]$project
[1] "en.wikipedia"

$items[[3]]$article
[1] "Aaron_Halfaker"

$items[[3]]$granularity
[1] "daily"

$items[[3]]$timestamp
[1] "2015100300"

$items[[3]]$access
[1] "all-access"

$items[[3]]$agent
[1] "all-agents"

$items[[3]]$views
[1] 10


$items[[4]]
$items[[4]]$project
[1] "en.wikipedia"

$items[[4]]$article
[1] "Aaron_Halfaker"

$items[[4]]$granularity
[1] "daily"

$items[[4]]$timestamp
[1] "2015100400"

$items[[4]]$access
[1] "all-access"

$items[[4]]$agent
[1] "all-agents"

$items[[4]]$views
[1] 11


$items[[5]]
$items[[5]]$project
[1] "en.wikipedia"

$items[[5]]$article
[1] "Aaron_Halfaker"

$items[[5]]$granularity
[1] "daily"

$items[[5]]$timestamp
[1] "2015100500"

$items[[5]]$access
[1] "all-access"

$items[[5]]$agent
[1] "all-agents"

$items[[5]]$views
[1] 8


$items[[6]]
$items[[6]]$project
[1] "en.wikipedia"

$items[[6]]$article
[1] "Aaron_Halfaker"

$items[[6]]$granularity
[1] "daily"

$items[[6]]$timestamp
[1] "2015100600"

$items[[6]]$access
[1] "all-access"

$items[[6]]$agent
[1] "all-agents"

$items[[6]]$views
[1] 17


$items[[7]]
$items[[7]]$project
[1] "en.wikipedia"

$items[[7]]$article
[1] "Aaron_Halfaker"

$items[[7]]$granularity
[1] "daily"

$items[[7]]$timestamp
[1] "2015100700"

$items[[7]]$access
[1] "all-access"

$items[[7]]$agent
[1] "all-agents"

$items[[7]]$views
[1] 13


$items[[8]]
$items[[8]]$project
[1] "en.wikipedia"

$items[[8]]$article
[1] "Aaron_Halfaker"

$items[[8]]$granularity
[1] "daily"

$items[[8]]$timestamp
[1] "2015100800"

$items[[8]]$access
[1] "all-access"

$items[[8]]$agent
[1] "all-agents"

$items[[8]]$views
[1] 4


$items[[9]]
$items[[9]]$project
[1] "en.wikipedia"

$items[[9]]$article
[1] "Aaron_Halfaker"

$items[[9]]$granularity
[1] "daily"

$items[[9]]$timestamp
[1] "2015100900"

$items[[9]]$access
[1] "all-access"

$items[[9]]$agent
[1] "all-agents"

$items[[9]]$views
[1] 6


$items[[10]]
$items[[10]]$project
[1] "en.wikipedia"

$items[[10]]$article
[1] "Aaron_Halfaker"

$items[[10]]$granularity
[1] "daily"

$items[[10]]$timestamp
[1] "2015101000"

$items[[10]]$access
[1] "all-access"

$items[[10]]$agent
[1] "all-agents"

$items[[10]]$views
[1] 8


$items[[11]]
$items[[11]]$project
[1] "en.wikipedia"

$items[[11]]$article
[1] "Aaron_Halfaker"

$items[[11]]$granularity
[1] "daily"

$items[[11]]$timestamp
[1] "2015101100"

$items[[11]]$access
[1] "all-access"

$items[[11]]$agent
[1] "all-agents"

$items[[11]]$views
[1] 3


$items[[12]]
$items[[12]]$project
[1] "en.wikipedia"

$items[[12]]$article
[1] "Aaron_Halfaker"

$items[[12]]$granularity
[1] "daily"

$items[[12]]$timestamp
[1] "2015101200"

$items[[12]]$access
[1] "all-access"

$items[[12]]$agent
[1] "all-agents"

$items[[12]]$views
[1] 7


$items[[13]]
$items[[13]]$project
[1] "en.wikipedia"

$items[[13]]$article
[1] "Aaron_Halfaker"

$items[[13]]$granularity
[1] "daily"

$items[[13]]$timestamp
[1] "2015101300"

$items[[13]]$access
[1] "all-access"

$items[[13]]$agent
[1] "all-agents"

$items[[13]]$views
[1] 7


$items[[14]]
$items[[14]]$project
[1] "en.wikipedia"

$items[[14]]$article
[1] "Aaron_Halfaker"

$items[[14]]$granularity
[1] "daily"

$items[[14]]$timestamp
[1] "2015101400"

$items[[14]]$access
[1] "all-access"

$items[[14]]$agent
[1] "all-agents"

$items[[14]]$views
[1] 4


$items[[15]]
$items[[15]]$project
[1] "en.wikipedia"

$items[[15]]$article
[1] "Aaron_Halfaker"

$items[[15]]$granularity
[1] "daily"

$items[[15]]$timestamp
[1] "201510

### Rate-limiting
The next stage of respectful API usage is rate-limiting: making sure you only make a certain number of requests to the server in a given time period. Your limit will vary from server to server, but the implementation is always pretty much the same and involves a call to Sys.sleep(). This function takes one argument, a number, which represents the number of seconds to "sleep" (pause) the R session for. So if you call Sys.sleep(15), it'll pause for 15 seconds before allowing further code to run.

As you can imagine, this is really useful for rate-limiting. If you are only allowed 4 requests a minute? No problem! Just pause for 15 seconds between each request and you're guaranteed to never exceed it. Let's demonstrate now by putting together a little loop that sends multiple requests on a 5-second time delay. We'll use httpbin.org 's APIs, which allow you to test different HTTP libraries.

In [11]:
# Construct a vector of 2 URLs
urls <- c("http://httpbin.org/status/404", "http://httpbin.org/status/301" )

for(url in urls){
    # Send a GET request to url
    result <- GET(url)
    # Delay for 5 seconds between requests
    Sys.sleep(5)
}

### Tying it all together
Using everything that you learned in the chapter, let's make a simple replica of one of the 'pageviews' functions - building queries, sending GET requests (with an appropriate user agent) and handling the output in a fault-tolerant way. You'll build this function up piece by piece in this exercise.

To output an error, you will use the function stop(), which takes a string as an argument, stops the execution of the program, and outputs the string as an error. You can try it right now by running stop("This is an error")

In [16]:
get_pageviews <- function(article_title){
  url <- paste(
    "https://wikimedia.org/api/rest_v1/metrics/pageviews/per-article/en.wikipedia/all-access/all-agents", 
    article_title, 
    "daily/2015100100/2015103100", 
    sep = "/"
  )   
  response <- GET(url, user_agent("my@email.com this is a test")) 
  # Is there an HTTP error?
  if(http_error(response)){ 
    # Throw an R error
    stop("the request failed") 
  }
  # Return the response's content
  content(response)
}

# run 
get_pageviews("Aaron_Halfaker")
# check error
get_pageviews("fake_article")


$items
$items[[1]]
$items[[1]]$project
[1] "en.wikipedia"

$items[[1]]$article
[1] "Aaron_Halfaker"

$items[[1]]$granularity
[1] "daily"

$items[[1]]$timestamp
[1] "2015100100"

$items[[1]]$access
[1] "all-access"

$items[[1]]$agent
[1] "all-agents"

$items[[1]]$views
[1] 8


$items[[2]]
$items[[2]]$project
[1] "en.wikipedia"

$items[[2]]$article
[1] "Aaron_Halfaker"

$items[[2]]$granularity
[1] "daily"

$items[[2]]$timestamp
[1] "2015100200"

$items[[2]]$access
[1] "all-access"

$items[[2]]$agent
[1] "all-agents"

$items[[2]]$views
[1] 3


$items[[3]]
$items[[3]]$project
[1] "en.wikipedia"

$items[[3]]$article
[1] "Aaron_Halfaker"

$items[[3]]$granularity
[1] "daily"

$items[[3]]$timestamp
[1] "2015100300"

$items[[3]]$access
[1] "all-access"

$items[[3]]$agent
[1] "all-agents"

$items[[3]]$views
[1] 10


$items[[4]]
$items[[4]]$project
[1] "en.wikipedia"

$items[[4]]$article
[1] "Aaron_Halfaker"

$items[[4]]$granularity
[1] "daily"

$items[[4]]$timestamp
[1] "2015100400"

$items[[4]]$access
[1] "all-access"

$items[[4]]$agent
[1] "all-agents"

$items[[4]]$views
[1] 11


$items[[5]]
$items[[5]]$project
[1] "en.wikipedia"

$items[[5]]$article
[1] "Aaron_Halfaker"

$items[[5]]$granularity
[1] "daily"

$items[[5]]$timestamp
[1] "2015100500"

$items[[5]]$access
[1] "all-access"

$items[[5]]$agent
[1] "all-agents"

$items[[5]]$views
[1] 8


$items[[6]]
$items[[6]]$project
[1] "en.wikipedia"

$items[[6]]$article
[1] "Aaron_Halfaker"

$items[[6]]$granularity
[1] "daily"

$items[[6]]$timestamp
[1] "2015100600"

$items[[6]]$access
[1] "all-access"

$items[[6]]$agent
[1] "all-agents"

$items[[6]]$views
[1] 17


$items[[7]]
$items[[7]]$project
[1] "en.wikipedia"

$items[[7]]$article
[1] "Aaron_Halfaker"

$items[[7]]$granularity
[1] "daily"

$items[[7]]$timestamp
[1] "2015100700"

$items[[7]]$access
[1] "all-access"

$items[[7]]$agent
[1] "all-agents"

$items[[7]]$views
[1] 13


$items[[8]]
$items[[8]]$project
[1] "en.wikipedia"

$items[[8]]$article
[1] "Aaron_Halfaker"

$items[[8]]$granularity
[1] "daily"

$items[[8]]$timestamp
[1] "2015100800"

$items[[8]]$access
[1] "all-access"

$items[[8]]$agent
[1] "all-agents"

$items[[8]]$views
[1] 4


$items[[9]]
$items[[9]]$project
[1] "en.wikipedia"

$items[[9]]$article
[1] "Aaron_Halfaker"

$items[[9]]$granularity
[1] "daily"

$items[[9]]$timestamp
[1] "2015100900"

$items[[9]]$access
[1] "all-access"

$items[[9]]$agent
[1] "all-agents"

$items[[9]]$views
[1] 6


$items[[10]]
$items[[10]]$project
[1] "en.wikipedia"

$items[[10]]$article
[1] "Aaron_Halfaker"

$items[[10]]$granularity
[1] "daily"

$items[[10]]$timestamp
[1] "2015101000"

$items[[10]]$access
[1] "all-access"

$items[[10]]$agent
[1] "all-agents"

$items[[10]]$views
[1] 8


$items[[11]]
$items[[11]]$project
[1] "en.wikipedia"

$items[[11]]$article
[1] "Aaron_Halfaker"

$items[[11]]$granularity
[1] "daily"

$items[[11]]$timestamp
[1] "2015101100"

$items[[11]]$access
[1] "all-access"

$items[[11]]$agent
[1] "all-agents"

$items[[11]]$views
[1] 3


$items[[12]]
$items[[12]]$project
[1] "en.wikipedia"

$items[[12]]$article
[1] "Aaron_Halfaker"

$items[[12]]$granularity
[1] "daily"

$items[[12]]$timestamp
[1] "2015101200"

$items[[12]]$access
[1] "all-access"

$items[[12]]$agent
[1] "all-agents"

$items[[12]]$views
[1] 7


$items[[13]]
$items[[13]]$project
[1] "en.wikipedia"

$items[[13]]$article
[1] "Aaron_Halfaker"

$items[[13]]$granularity
[1] "daily"

$items[[13]]$timestamp
[1] "2015101300"

$items[[13]]$access
[1] "all-access"

$items[[13]]$agent
[1] "all-agents"

$items[[13]]$views
[1] 7


$items[[14]]
$items[[14]]$project
[1] "en.wikipedia"

$items[[14]]$article
[1] "Aaron_Halfaker"

$items[[14]]$granularity
[1] "daily"

$items[[14]]$timestamp
[1] "2015101400"

$items[[14]]$access
[1] "all-access"

$items[[14]]$agent
[1] "all-agents"

$items[[14]]$views
[1] 4


$items[[15]]
$items[[15]]$project
[1] "en.wikipedia"

$items[[15]]$article
[1] "Aaron_Halfaker"

$items[[15]]$granularity
[1] "daily"

$items[[15]]$timestamp
[1] "201510

ERROR: Error in get_pageviews("fake_article"): the request failed
